In [118]:
import cv2
import pytesseract
import pandas as pd
import re
import matplotlib.pyplot as plt
from PIL import Image
import os 

# Charger l'image
image_path = "../data/page_1.png"
image = cv2.imread(image_path)

image

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [119]:
def extract_informations(text):
    
    """
    Fonction pour extraire les noms, prénoms et sujets d'un texte brut.
    """

    # Séparer le texte en lignes
    lines = text.split("\n")

    # Expression régulière pour détecter un nom avec le motif (NOM)
    pattern = re.compile(r"([A-ZÉÀÈÙÂÊÎÔÛÄËÏÖÜÇ]{2,}) \(([^)]+)\)\. (.+)")

    # Stocker les résultats
    data = []

    for line in lines:
        
        

        match = pattern.search(line)
        if match:
            nom = match.group(1)
            prenom = match.group(2)
            sujet = match.group(3)
            data.append([nom, prenom, sujet])

    # Création du DataFrame
    df = pd.DataFrame(data, columns=["Nom", "Prénom", "Sujet"])

    return df

In [120]:
def process_image(image) : 
    # Convertir en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)

    # Récupérer les dimensions
    (h, w) = gray.shape

    # Scinder en deux parties
    left_part = gray[:, :w//2 - 5]   # Partie gauche
    right_part = gray[:,  5 + w//2:]  # Partie droite

    left_text = pytesseract.image_to_string(left_part, lang = 'fra')
    right_text = pytesseract.image_to_string(right_part, lang = 'fra')

    df_left = extract_informations(left_text)
    df_right = extract_informations(right_text)

    df_final = pd.concat([df_left, df_right], ignore_index=True)  # ignore_index évite la duplication des index

    return df_final


In [123]:
def process_folder(input_dir):
    # Generate a timestamped folder 
    df = pd.DataFrame()
    
    for year_folder in sorted(os.listdir(input_dir)):
        year_path = os.path.join(input_dir, year_folder)

        if os.path.isdir(year_path):  # Check if it's a directory

            for image_file in (os.listdir(year_path)):
                image_path = os.path.join(year_path, image_file)

                if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image formats
                    print(f"Processing {image_path}...")
                    df = pd.concat(df, process_image(image), ignore_index= True)

    return df

current_dir = os.getcwd()

input_dir = os.path.join(current_dir, '..', 'data', 'pdfs_en_images_png', '1870')  # Relative path to 'lib/data/pdfs_en_images_png'
output_dir = os.path.join(current_dir, '..', 'data', 'texte_extrait_par_pdfs')  # Output path within 'texte_extrait_par_pdfs'



# print(process_folder(input_dir))

input_dir


'/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870'